In [7]:
import pandas as pd
import os
from tqdm import trange
import requests
import json
from typing import List, Dict, Tuple, Any
import time


## load env variables
from dotenv import load_dotenv

load_dotenv()
HF_KEY = os.environ['HUGGINGFACE']
API_URL = os.environ['HF_API_URL']


In [3]:
def load_region_data(region: str) -> pd.DataFrame:
    # Define the file path based on the region
    region_path = f"../data/official_data/feedback_{region}.xlsx"

    # Specify columns to read
    columns_to_read = ["Feedback id", "Feedback 1", "Feedback 2", "URL"]

    # Load the data into a DataFrame
    df = pd.read_excel(region_path, usecols=columns_to_read)

    # Filter out rows with missing or invalid data
    df_filtered = df[
        (df['Feedback 1'].notna()) &
        (df['Feedback 2'].notna()) &
        (df['Feedback 2'] != '{"description":""}')
    ].copy()  # Ensure df_filtered is a separate copy

    # Extract the 'description' field from JSON in 'Feedback 2'
    df_filtered.loc[:, 'Feedback 2'] = df_filtered['Feedback 2'].apply(
        lambda x: json.loads(x)['description'] if isinstance(x, str) else None
    )

    # Convert 'Feedback id' to numeric and drop rows with invalid IDs
    df_filtered.loc[:, 'Feedback id'] = pd.to_numeric(df_filtered['Feedback id'], errors='coerce')
    df_filtered = df_filtered.dropna(subset=['Feedback id'])
    df_filtered.reset_index(drop=True, inplace=True)

    return df_filtered

def translate_batch(region_lowercased: str, foreign_texts: list):
    payload = {
        "inputs": [text for text in foreign_texts],
        "parameters": {"src_lang": region_lowercased,
                        "tgt_lang": "en"}
    }
    
    response = requests.post(
        API_URL, headers={"Authorization": f"Bearer {HF_KEY}"}, json=payload
    )    
    output = response.json()

    if 'error' in output:
        raise Exception(output['error'])
    
    translated_text = [{"Original": foreign_texts[i],
                        "Translated" : output[i]['translation_text'][3:]} for i in range(len(output))]
    
    return translated_text

def format_llm_input(df: pd.DataFrame) -> Tuple[List[Dict[str, str]], Dict[int, str]]:
    # Extract feedback IDs and feedback text
    feedback_ids = list(df['Feedback id'])
    feedback_texts = list(df['Feedback 2'])

    # Create a dictionary mapping feedback IDs to feedback text
    id_feedback = {int(feedback_id): feedback for feedback_id, feedback in zip(feedback_ids, feedback_texts)}

    # Prepare the LLM input as a list of dictionaries
    llm_input = [{'id': feedback_id, 'feedback': feedback} for feedback_id, feedback in id_feedback.items()]

    return llm_input, id_feedback

In [6]:
region  = "VN_Article"
df = load_region_data(region)
llm_input, id_feedback = format_llm_input(df)
llm_input



c:\Users\SP14267\Desktop\seh\venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


[{'id': 3558716, 'feedback': 'Bạn có thể khoá luôn tk gian hàng của tôi nhé'},
 {'id': 3557525,
  'feedback': 'Không hướng dẫn trên bảng điện thoại hay máy máy rõ ràng, tìm hoài không ra'},
 {'id': 3554240, 'feedback': 'Tôi là người mua'},
 {'id': 3552371, 'feedback': 'mình không hủy đuoc dịch vụ freeship extra.'},
 {'id': 3552236,
  'feedback': 'Tôi vào tài khoản của tôi không có mục Công cụ thiết kế hình ảnh !'},
 {'id': 3550988,
  'feedback': 'không có thông tin như trong hình ảnh hướng dẫn'},
 {'id': 3550622,
  'feedback': 'bản dùng cho web có không, tại tôi thấy hướng dẫn cho app ak k thấy hướng dẫn cho WEb\n'},
 {'id': 3550497,
  'feedback': 'Lý giải chi tiết hơn bằng video hướng dẫn giải thích cụ thể vì sao'},
 {'id': 3549241, 'feedback': 'Gọi không được'},
 {'id': 3547268, 'feedback': 'Bỏ rồi à?'},
 {'id': 3545071,
  'feedback': 'hàng giả thì nhan nhãn tệ hại lấy cái giá hàng giả đòi đấu giá lạy hồn. thiết kế chương trình kiểu gì ko biết. Hàng nhà máy còn chả có giá đó'},
 {'id

In [ ]:
# Loop through the llm_input
def translate_region_text(llm_input, region_lowercased):
    window = 2
    for i in range(0,len(llm_input), window):
        batch = 
        res = translate_batch(region_lowercased, batch)
        res
    return

In [39]:
region_lowercased = region.lower()
sample_text_1 = "Hãy luôn tin tưởng vào bản thân, vì bạn mạnh mẽ hơn bạn nghĩ."
sample_text_2 = "Những điều nhỏ bé hàng ngày tạo nên sự khác biệt lớn trong cuộc sống."
foreign_texts = [sample_text_1, sample_text_2]



c:\Users\SP14267\Desktop\seh\venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


[{'Original': 'Hãy luôn tin tưởng vào bản thân, vì bạn mạnh mẽ hơn bạn nghĩ.',
  'Translated': 'always believe in yourself, because you are stronger than you think.'},
 {'Original': 'Những điều nhỏ bé hàng ngày tạo nên sự khác biệt lớn trong cuộc sống.',
  'Translated': "Every day's little things make a big difference in life."}]